In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import jieba
import jieba.analyse

In [2]:
from bs4 import BeautifulSoup 
import requests
# 取得頁面原始碼
def getPageContent(URL):
    res = requests.get(
        url=URL,
        cookies={'over18':'1'}
    )
    content = BeautifulSoup(res.text, 'html.parser')
    return content

In [3]:
# 取得該版文章總頁數
def getPageLinks(start_page, end_page):
    #要爬的起始頁數 & 總頁數
    
    links = []
    for i in range(start_page, end_page, -1):
        link = 'https://www.ptt.cc/bbs/HatePolitics/index' + str(i) + '.html'
        links.append(link)
        
    return links

In [4]:
# 取得目前文章列表頁的文章連結
def getEachRow(URL):
    content = getPageContent(URL)
    rows = content.findAll('div',{'class':'r-ent'})
    
    link = []
    title = []
    date = []
    author = []
    push = []
    
    for row in rows:
        meta = row.find('div', 'title').find('a')
        if meta :             # 如果文章已被刪除，meta 會是 None
            link.append(meta['href'])
            title.append(meta.text)
            date.append(row.find('div', 'date').text)
            author.append(row.find('div', 'author').text)
            push.append(row.find('div', 'nrec').text)
    article_df = pd.DataFrame({'Date': date, 'Push': push, 'Title': title, 'Author': author, 'Link': link})
    return article_df

In [5]:
# 用該文章連結取得文章內文
def getArticle(URL):
    article = getPageContent(URL)
    information = article.findAll('div', ['richcontent', 'article-metaline', 'article-metaline-right', 'push'])
    information2 = article.findAll('a')
    edit = article.findAll('span', 'f2')
    #刪除不要的標籤
    for i in information:
        i = i.extract()
    for i in information2:
        i = i.extract()
    for i in edit:
        i = i.extract()
    #取得內文
    main_content = article.find(id = 'main-content')
    if main_content:
        article = main_content.text.lstrip().rstrip()
    else:
        article = 'None'
    return article

In [6]:
links = getPageLinks(3418, 763)
all_post_meta = pd.DataFrame(columns=['Date', 'Push', 'Title', 'Author', 'Link'])
for link in links:
    df = getEachRow(link)
    df = df.iloc[::-1]
    all_post_meta = pd.concat([all_post_meta, df], ignore_index = True, sort = False)

In [17]:
#爬取內文，並併入Dataframe
contents = []
count = 0
for post_meta in all_post_meta['Link']:
    content = getArticle('https://www.ptt.cc'+ post_meta)
    contents.append(content)
#     count += 1
#     if count % 2500 == 0:
#         print(time.ctime())

Wed May 22 22:51:24 2019
Wed May 22 23:13:44 2019
Wed May 22 23:33:01 2019
Wed May 22 23:47:41 2019
Thu May 23 00:01:28 2019
Thu May 23 00:15:33 2019
Thu May 23 00:29:46 2019
Thu May 23 00:43:56 2019
Thu May 23 00:57:05 2019
Thu May 23 01:10:42 2019
Thu May 23 01:23:52 2019
Thu May 23 01:38:01 2019
Thu May 23 01:51:19 2019
Thu May 23 02:04:19 2019
Thu May 23 02:16:53 2019
Thu May 23 02:29:51 2019
Thu May 23 02:42:29 2019
Thu May 23 02:55:02 2019
Thu May 23 03:07:14 2019
Thu May 23 03:19:31 2019
Thu May 23 03:31:44 2019


In [49]:
# content_df = pd.DataFrame({'Content': contents})
# all_post_meta = pd.concat([all_post_meta, content_df], axis = 1, ignore_index = False, sort = False) #axis = 1, 橫向合併
# all_post_meta.to_csv('post.csv', encoding='utf_8_sig')

In [3]:
all_post_meta = pd.read_csv('post.csv', index_col = 0)

In [4]:
#去除頭尾日期
all_post_meta = all_post_meta[all_post_meta['Date'].str.contains('12/31') == False]
all_post_meta = all_post_meta[all_post_meta['Date'].str.contains('5/01') == False]
all_post_meta = all_post_meta.reset_index(drop = True) #reset index after drop rows
all_post_meta

,Date,Push,Title,Author,Link,Content
0,4/30,27,[轉錄] 韓粉不要看！,littlered112,/bbs/HatePolitics/M.1556639935.A.E03.html,1.轉錄標題︰\r\n\r\n【Yahoo論壇 / 唐湘龍】韓粉不要看！\r\n\r\n2....
1,4/30,4,Fw: [問卦] 因為有戒嚴，以前的台灣治安才會那麼好。,oftisa,/bbs/HatePolitics/M.1556639780.A.C62.html,放屁\r\n\r\n戒嚴時代黑道就是中華民國/中國國民黨養的\r\n\r\n戒嚴時期應該正名...
2,4/30,25,Re: [討論] 郭跟川普差在哪兒？,omanorboyo,/bbs/HatePolitics/M.1556639770.A.2BE.html,郭董絕對是今天經濟論壇整排來賓口條最差的 感覺是想啥講啥\r\n\r\n話很常黏在一起 講話...
3,4/30,NaN,[討論] 政黑真的很多假韓粉。,brian21201,/bbs/HatePolitics/M.1556639746.A.F0A.html,政黑真的被滲透了!\r\n\r\n明明之前很多人挺老韓的啊\r\n\r\n為什麼現在連一篇澄...
4,4/30,13,[討論] 怎麼一個晚上韓總聲勢整個大翻轉？,ray90910,/bbs/HatePolitics/M.1556639689.A.D41.html,小弟今天早上看一下板上\r\n韓總還是不少人捧\r\n很多人也覺得大概就是韓出線了\r\n今...
5,4/30,5,[討論] 大家還記得噁心大人嗎？,momo28,/bbs/HatePolitics/M.1556639645.A.5B6.html,大家知道他現在在那服務嗎??\r\n\r\n答案是\r\n\r\n\r\n\r\n\r\n\...
6,4/30,1,Re: [討論] 郭跟川普差在哪兒？,sading7,/bbs/HatePolitics/M.1556639627.A.DA0.html,川普和果凍的差別是很明顯的\r\n\r\n一個是美國優先，一個是中國優先\r\n\r\n川普...
7,4/30,3,Re: [轉錄] 韓國瑜逐字稿-如何使台灣安全及統獨分析,coolhon,/bbs/HatePolitics/M.1556639603.A.CA1.html,韓國瑜的話不用討論太多，討論太多只是顯得你愚蠢而已\r\n\r\n韓喜歡講一些似是而非的話，...
8,4/30,31,[討論] 韓國瑜為啥可以這麼討厭？,rayterzll,/bbs/HatePolitics/M.1556639310.A.AFC.html,如題\r\n\r\n\r\n那張嘴、那個智商、那個自大、那個膨脹\r\n\r\n\r\n當年...
9,4/30,3,[討論] 台灣的防火牆 不要有一黨獨大情況,XX9,/bbs/HatePolitics/M.1556639237.A.4D0.html,台灣目前小黨派已經有很多選項了\r\n\r\n這次我認為 只要國民黨推出來的不是草包\r\n...


In [5]:
#取出四月post
all_post_meta = all_post_meta[all_post_meta['Date'].str.contains('4/') == True]
all_post_meta = all_post_meta.reset_index(drop = True) #reset index after drop rows
all_post_meta

,Date,Push,Title,Author,Link,Content
0,4/30,27,[轉錄] 韓粉不要看！,littlered112,/bbs/HatePolitics/M.1556639935.A.E03.html,1.轉錄標題︰\r\n\r\n【Yahoo論壇 / 唐湘龍】韓粉不要看！\r\n\r\n2....
1,4/30,4,Fw: [問卦] 因為有戒嚴，以前的台灣治安才會那麼好。,oftisa,/bbs/HatePolitics/M.1556639780.A.C62.html,放屁\r\n\r\n戒嚴時代黑道就是中華民國/中國國民黨養的\r\n\r\n戒嚴時期應該正名...
2,4/30,25,Re: [討論] 郭跟川普差在哪兒？,omanorboyo,/bbs/HatePolitics/M.1556639770.A.2BE.html,郭董絕對是今天經濟論壇整排來賓口條最差的 感覺是想啥講啥\r\n\r\n話很常黏在一起 講話...
3,4/30,NaN,[討論] 政黑真的很多假韓粉。,brian21201,/bbs/HatePolitics/M.1556639746.A.F0A.html,政黑真的被滲透了!\r\n\r\n明明之前很多人挺老韓的啊\r\n\r\n為什麼現在連一篇澄...
4,4/30,13,[討論] 怎麼一個晚上韓總聲勢整個大翻轉？,ray90910,/bbs/HatePolitics/M.1556639689.A.D41.html,小弟今天早上看一下板上\r\n韓總還是不少人捧\r\n很多人也覺得大概就是韓出線了\r\n今...
5,4/30,5,[討論] 大家還記得噁心大人嗎？,momo28,/bbs/HatePolitics/M.1556639645.A.5B6.html,大家知道他現在在那服務嗎??\r\n\r\n答案是\r\n\r\n\r\n\r\n\r\n\...
6,4/30,1,Re: [討論] 郭跟川普差在哪兒？,sading7,/bbs/HatePolitics/M.1556639627.A.DA0.html,川普和果凍的差別是很明顯的\r\n\r\n一個是美國優先，一個是中國優先\r\n\r\n川普...
7,4/30,3,Re: [轉錄] 韓國瑜逐字稿-如何使台灣安全及統獨分析,coolhon,/bbs/HatePolitics/M.1556639603.A.CA1.html,韓國瑜的話不用討論太多，討論太多只是顯得你愚蠢而已\r\n\r\n韓喜歡講一些似是而非的話，...
8,4/30,31,[討論] 韓國瑜為啥可以這麼討厭？,rayterzll,/bbs/HatePolitics/M.1556639310.A.AFC.html,如題\r\n\r\n\r\n那張嘴、那個智商、那個自大、那個膨脹\r\n\r\n\r\n當年...
9,4/30,3,[討論] 台灣的防火牆 不要有一黨獨大情況,XX9,/bbs/HatePolitics/M.1556639237.A.4D0.html,台灣目前小黨派已經有很多選項了\r\n\r\n這次我認為 只要國民黨推出來的不是草包\r\n...


In [13]:
# article_df = getEachRow('https://www.ptt.cc/bbs/HatePolitics/index764.html')
# article_df = article_df[article_df['Date'] != '12/31']
# article_df.reset_index(drop=True, inplace=True)
# article_df

,Date,Push,Title,Author,Link
0,1/01,31,[黑特] 2019 新希望,Neptunium,/bbs/HatePolitics/M.1546272003.A.8D2.html
1,1/01,X3,[黑特] 2019年第一篇批柯文！！,KyleSeager,/bbs/HatePolitics/M.1546272014.A.EAD.html
2,1/01,3,Re: [討論] 台北和高雄的跨年這次誰獲勝？,ceca,/bbs/HatePolitics/M.1546272438.A.B12.html
3,1/01,8,[討論] 高雄沒煙火?,s49971616,/bbs/HatePolitics/M.1546272475.A.C61.html
4,1/01,爆,[討論] 台北跟高雄這次跨年 誰比較厲害好看?,ruby00514,/bbs/HatePolitics/M.1546272560.A.86E.html
5,1/01,,[討論] 柯P選總統?,raymond5566,/bbs/HatePolitics/M.1546272994.A.643.html
6,1/01,22,[黑特] 幹 比什麼比,alberchi,/bbs/HatePolitics/M.1546273275.A.518.html
7,1/01,11,[新聞] 葛來儀否認邀柯文哲訪美 指柯幕僚扭曲事,pfry19855,/bbs/HatePolitics/M.1546273338.A.190.html
8,1/01,X1,[討論] 高雄人有沒有後悔了,ianmon,/bbs/HatePolitics/M.1546273741.A.D4C.html
9,1/01,40,[黑特] 高雄今晚是不是雙輸,momo28,/bbs/HatePolitics/M.1546273906.A.733.html


In [7]:
word_list = []

content = all_post_meta['Content']
content_list = list(content)

In [8]:
#import self_defined stop_list
stop_list = [line.strip() for line in open('停用詞.txt','r+', encoding = 'utf-8')]
stop_list[0] = '$'
stop_list.extend(['\n', '\r', ' ', '/', ':', '#', '-', '%', 'nan', '【', '】', '(', ')', '=', ',', '&', '\u3000'])

In [10]:
for stopword in stop_list:
    content_list = [word.replace(stopword, '') for word in content_list]

content_list

['轉錄標題︰Yahoo壇唐湘龍韓粉轉錄網址︰轉錄容︰粉快速變黑技巧近發現變黑黑黑黑晚穿衣服出門黑想乳液防曬係數夠SPF太低近韓粉強光射太久紫外線太強整黑掉坦白說沒底白紅黑周期性變黑尤年反太陽花學運反洪仲丘事件反核四反課綱反柯文哲反民進黨…路反路黑年烏骨雞黑毛豬黑骨頭裡直韓國瑜出現韓流崛九合選舉勝翻身終站陽光韓粉剛選市長二天群神病徵召韓國瑜選總統天聽韓國瑜算中痛韓國瑜算中痛知道什韓國瑜算中痛樣算痛反正媒體毫政治倫理造神韓粉近歇斯底里期烏鴉烏鴉黑種重選舉作線遊戲破關關忍住發出微弱質疑冷聲音然走黑道路間沒延伸關係沒相互佔權利黎明混夜色時淺淺重疊片刻白天黑夜交沒交換法想方世界堅持原站成兩世界永遠懂傷悲白天懂夜黑永恆燃燒太陽懂月亮盈缺永遠懂傷悲白天懂夜黑首歌白天懂夜黑英唱眼淚唱請賓鼓勵鼓勵藍軍網路蓋布袋霸凌私刑忠貞二然黨員總演入戲國民黨間殺器喜歡國旗熱愛國家歷史情感陪伴苦難民族路走核心價值覺指紋密碼臉部辨識視網膜掃描…應開機應韓粉抱取暖殺四方保衛中華民國敢批評韓國瑜殺殺殺殺殺殺殺立七殺碑沒想竟然殺師性乖戾心陰暗常常喜歡指責政治判斷力太差實差白痴年伍子胥昭關夜間鬚髮皆白唐湘龍反韓流夜間全身發黑怕快死突然間公審罪名罪知道說韓國瑜沒幫丁守中站台丁守中輸懷恨心明明說郭台銘參選挺郭台銘明明郭台銘先生面緣然見笑轉生氣發出宰前豬樣淒厲哀號豬年豬年權利現終知道古什說高處勝韓實覺韓國瑜變變變容莊敬強處變驚變應萬變知道韓粉辦辦認識兩韓國瑜年前韓國瑜九合選舉韓國瑜年前實國民黨裡面衝組罵罵動手動腳韓國瑜見九合選舉出現韓國瑜外型風格完全面民進黨府院黨鋪天蓋追殺抹黑黃抹紅總愛包容出惡言守陽光守真迷透韓粉變成韓黑然粉黑粉黑純粹近發現原兩韓國瑜韓國瑜黨初選場現沒表態參選初選年前韓國瑜回美國隔海批評馬英九馬英九然批評韓國瑜批評帶輕蔑屑馬英九天拉臉邀請韓國瑜參加濟壇韓國瑜批評黨初選權貴密室協商更失厚道路提拔韓國瑜溺愛韓國瑜推出強韓國瑜國民黨放棄候選感受政黨原黨志厚愛志充滿懷疑攻擊中華民國敵完全愛包容陳水扁馬英九更樣變韓國瑜變沒關係實變回變變變變變果韓國瑜夠真正變成團結藍軍共主算黑粉黑什關係講完廢話應難理解近什變黑活國民黨黨初選屬意候選定樣沒關係初選團結粉水泥臉皮厚做附註心想法︰唐湘龍段話感觸韓粉韓國瑜真變',
 '放屁戒嚴時代黑道中華民國中國國民黨養戒嚴時期應正名黨國獨裁時期黨國獨裁政府想幹幹外省幫黑道橫行外

In [11]:
content_word = []
for i in range(len(content_list)):
    content_word.append([])
    for k in jieba.cut(content_list[i].strip(), cut_all = False):
        content_word[i].append(k)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Perry\AppData\Local\Temp\jieba.cache
Loading model cost 1.249 seconds.
Prefix dict has been built succesfully.


In [15]:
content_word1 = content_word[:5]
content_word1[4]

['弟天',
 '早板',
 '韓總少',
 '捧',
 '覺概',
 '韓出',
 '線天',
 '晚弟',
 '絢輝龍',
 '直現',
 '睡',
 '前板',
 '韓總',
 '負評',
 '洗板',
 '逐字',
 '稿',
 '太',
 '懂',
 '解釋',
 '韓國瑜',
 '突然',
 '負評',
 '潮龍脈',
 'SentfromJPTTonmyHMDGlobalNokia']

In [35]:
import gensim
from gensim import corpora, models, similarities

id2word = corpora.Dictionary(content_word1)
corpus = [dic.doc2bow(text) for text in content_word1]
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)

In [36]:
print(lda_model.print_topics())

[(0, '0.018*"獨裁" + 0.015*"戒嚴" + 0.015*"治安" + 0.015*"外省" + 0.013*"黨國" + 0.010*"黑道" + 0.010*"台灣" + 0.010*"新聞" + 0.010*"日子" + 0.008*"殺"'), (1, '0.018*"東坡" + 0.014*"曰" + 0.010*"哥哥" + 0.010*"佛印" + 0.010*"狗屎" + 0.010*"韓導" + 0.010*"佛" + 0.005*"想" + 0.005*"新聞" + 0.005*"年"'), (2, '0.002*"獨裁" + 0.002*"韓國瑜" + 0.002*"外省" + 0.002*"說" + 0.002*"治安" + 0.002*"台灣" + 0.002*"新聞" + 0.002*"韓粉" + 0.002*"戒嚴" + 0.002*"殺"'), (3, '0.024*"韓國瑜" + 0.010*"殺" + 0.010*"韓粉" + 0.009*"︰" + 0.009*"說" + 0.009*"選舉" + 0.009*"立委" + 0.008*"懂" + 0.008*"黨" + 0.006*"批評"')]


In [12]:
# 用該文章連結取得文章所有推文
def getPush(URL):
    content = getPageContent(URL)
    message = content.findAll('div', 'push')
    
#     userID = []
#     comment = []
    types = []
#     links = []
    
    for elem in message:
        if elem.find('span', 'f1 hl push-tag'):
            types.append(elem.find('span', 'f1 hl push-tag').text.strip())
        if elem.find('span', 'hl push-tag'):
            types.append(elem.find('span', 'hl push-tag').text.strip())
    
    push = pd.DataFrame({'Type': types})

    return push

In [69]:
resNum = []
pushNum = []
count = 0

for link in all_post_meta['Link']:
    types = getPush('https://www.ptt.cc'+ link)
    resNum.append(len(types)) #總回覆
    types = types[types['Type'].astype(str).str.contains('推')]
    pushNum.append(len(types)) #總推文
#     if resNum == 0:
#         pushRate.append(0)
#     else:
#         pushRate.append(pushNum/resNum)
    
    count += 1
    if count % 2500 == 0:
        print(time.ctime())


Thu May 30 01:56:04 2019
Thu May 30 02:07:42 2019
Thu May 30 02:19:09 2019
Thu May 30 02:30:37 2019
Thu May 30 02:42:03 2019
Thu May 30 02:53:39 2019
Thu May 30 03:05:03 2019


In [87]:
all_post_meta = pd.concat([all_post_meta, num], axis = 1, ignore_index = False, sort = False)

,Date,Push,Title,Author,Link,Content,PushNum,ResNum
0,4/30,27,[轉錄] 韓粉不要看！,littlered112,/bbs/HatePolitics/M.1556639935.A.E03.html,1.轉錄標題︰\r\n\r\n【Yahoo論壇 / 唐湘龍】韓粉不要看！\r\n\r\n2....,30,65
1,4/30,4,Fw: [問卦] 因為有戒嚴，以前的台灣治安才會那麼好。,oftisa,/bbs/HatePolitics/M.1556639780.A.C62.html,放屁\r\n\r\n戒嚴時代黑道就是中華民國/中國國民黨養的\r\n\r\n戒嚴時期應該正名...,4,13
2,4/30,25,Re: [討論] 郭跟川普差在哪兒？,omanorboyo,/bbs/HatePolitics/M.1556639770.A.2BE.html,郭董絕對是今天經濟論壇整排來賓口條最差的 感覺是想啥講啥\r\n\r\n話很常黏在一起 講話...,30,133
3,4/30,NaN,[討論] 政黑真的很多假韓粉。,brian21201,/bbs/HatePolitics/M.1556639746.A.F0A.html,政黑真的被滲透了!\r\n\r\n明明之前很多人挺老韓的啊\r\n\r\n為什麼現在連一篇澄...,0,2
4,4/30,13,[討論] 怎麼一個晚上韓總聲勢整個大翻轉？,ray90910,/bbs/HatePolitics/M.1556639689.A.D41.html,小弟今天早上看一下板上\r\n韓總還是不少人捧\r\n很多人也覺得大概就是韓出線了\r\n今...,13,29
5,4/30,5,[討論] 大家還記得噁心大人嗎？,momo28,/bbs/HatePolitics/M.1556639645.A.5B6.html,大家知道他現在在那服務嗎??\r\n\r\n答案是\r\n\r\n\r\n\r\n\r\n\...,7,10
6,4/30,1,Re: [討論] 郭跟川普差在哪兒？,sading7,/bbs/HatePolitics/M.1556639627.A.DA0.html,川普和果凍的差別是很明顯的\r\n\r\n一個是美國優先，一個是中國優先\r\n\r\n川普...,2,3
7,4/30,3,Re: [轉錄] 韓國瑜逐字稿-如何使台灣安全及統獨分析,coolhon,/bbs/HatePolitics/M.1556639603.A.CA1.html,韓國瑜的話不用討論太多，討論太多只是顯得你愚蠢而已\r\n\r\n韓喜歡講一些似是而非的話，...,4,8
8,4/30,31,[討論] 韓國瑜為啥可以這麼討厭？,rayterzll,/bbs/HatePolitics/M.1556639310.A.AFC.html,如題\r\n\r\n\r\n那張嘴、那個智商、那個自大、那個膨脹\r\n\r\n\r\n當年...,34,77
9,4/30,3,[討論] 台灣的防火牆 不要有一黨獨大情況,XX9,/bbs/HatePolitics/M.1556639237.A.4D0.html,台灣目前小黨派已經有很多選項了\r\n\r\n這次我認為 只要國民黨推出來的不是草包\r\n...,5,17


In [117]:
# han_post = all_post_meta[all_post_meta['Title'].str.contains('韓')]
# han_post = han_post.reset_index(drop = True) #reset index after drop rows
#四月韓國瑜的文章(用標題抓的)
all_post_meta.to_csv('April_post_meta.csv', index = False, encoding = 'utf_8_sig')
han_post.to_csv('han_post.csv', index = False, encoding = 'utf_8_sig')

In [115]:
han_post.head()

,Date,Push,Title,Author,Link,Content,PushNum,ResNum
0,4/30,27,[轉錄] 韓粉不要看！,littlered112,/bbs/HatePolitics/M.1556639935.A.E03.html,1.轉錄標題︰\r\n\r\n【Yahoo論壇 / 唐湘龍】韓粉不要看！\r\n\r\n2....,30,65
1,4/30,NaN,[討論] 政黑真的很多假韓粉。,brian21201,/bbs/HatePolitics/M.1556639746.A.F0A.html,政黑真的被滲透了!\r\n\r\n明明之前很多人挺老韓的啊\r\n\r\n為什麼現在連一篇澄...,0,2
2,4/30,13,[討論] 怎麼一個晚上韓總聲勢整個大翻轉？,ray90910,/bbs/HatePolitics/M.1556639689.A.D41.html,小弟今天早上看一下板上\r\n韓總還是不少人捧\r\n很多人也覺得大概就是韓出線了\r\n今...,13,29
3,4/30,3,Re: [轉錄] 韓國瑜逐字稿-如何使台灣安全及統獨分析,coolhon,/bbs/HatePolitics/M.1556639603.A.CA1.html,韓國瑜的話不用討論太多，討論太多只是顯得你愚蠢而已\r\n\r\n韓喜歡講一些似是而非的話，...,4,8
4,4/30,31,[討論] 韓國瑜為啥可以這麼討厭？,rayterzll,/bbs/HatePolitics/M.1556639310.A.AFC.html,如題\r\n\r\n\r\n那張嘴、那個智商、那個自大、那個膨脹\r\n\r\n\r\n當年...,34,77
